In [1]:
import os
import pandas as pd
import numpy as np

import nibabel as nib
from nilearn.image import clean_img,concat_imgs,math_img
from nilearn.plotting import plot_roi
from nilearn.input_data import NiftiMasker

from src.remove_vols import remove_specificvols
from src.mean_xvolumes import mean_xvolumes
from src.data_balancing import data_balancing
from src.MinMaxScaler_nifti import MinMaxScaler_nifti
from itertools import combinations

import warnings

warnings.filterwarnings("ignore")

c:\Users\anaga\anaconda3\lib\site-packages\nilearn\input_data\__init__.py:23: FutureWarning: The import path 'nilearn.input_data' is deprecated in version 0.9. Importing from 'nilearn.input_data' will be possible at least until release 0.13.0. Please import from 'nilearn.maskers' instead.
  warnings.warn(message, FutureWarning)


In [8]:
# Diretory where the events and image files are located
ROOT_DATA=r'D:\fmriprep'
RESULTS_DIR = r'C:\Users\anaga\OneDrive\Universidade\5º ano\Tese\results'

#Set variables
SES='ses-01'
TASK = 'task-01'
TR = 1

SUB='sub-01'
labels_type ='predef' #'predef' (predefined labels) or 'ptcps' (participant's labels)


In [9]:
# Data folder of each participant
data_path=os.path.join(ROOT_DATA, SUB, SES)
print('The data is in this folder - ' + data_path)


if labels_type == 'ptcps':

    #Data path to save the results
    results_path = os.path.join(RESULTS_DIR,'voxel_stability','participant_labels')

    #Diretory with the events files
    events_path = os.path.join(ROOT_DATA,'task-01permusicparticipant_run-1_events')

elif labels_type == 'predef':
    
    #Data path to save the results
    results_path = os.path.join(RESULTS_DIR,'voxel_stability','predefined_labels')

    #Diretory with the events files
    events_path = os.path.join(ROOT_DATA,'task-01permusicpredefined_run_events')


The data is in this folder - D:\fmriprep\sub-01\ses-01


In [10]:
# Pasta com os resultados de cada participante
SUB_results_path = os.path.join(results_path, SUB)

try:
    # Cria a pasta
    os.makedirs(SUB_results_path)
except FileExistsError:
    # Se a pasta já existir não acontece nada
    pass


## Prepare data

### Apply whole brain mask 

In [12]:
runs = [1,2,3,4]
for run in runs:
    RUN = 'run-'+str(run)

    file_name_format = SUB + '_' + SES + '_' + TASK + '_'+ RUN 
    print('Now computing ',file_name_format)
    

    # Load of preprocessed function image
    func_img_fn =os.path.join(data_path, 'func',file_name_format + '_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz')
    preprocessed_img = nib.load(func_img_fn)
    print('Functional image shape: ',preprocessed_img.shape)

    # Load of fmriprep whole-brain mask
    brain_mask_fn =os.path.join(data_path, 'func',file_name_format + '_space-MNI152NLin2009cAsym_desc-brain_mask.nii.gz')
    brain_mask = nib.load(brain_mask_fn)
    print('Brain mask image shape: ',brain_mask.shape)

    # Apply the mask to the functional image-> data matrix with (n_vols, n_voxels)
    nifti_masker = NiftiMasker(mask_img = brain_mask)
    masked_image_data = nifti_masker.fit_transform(preprocessed_img)
    print('masked_image_data.shape: ',masked_image_data.shape)

    # 4D Nifti image with the mask 
    masked_img = nifti_masker.inverse_transform(masked_image_data)
    print('masked_image.shape: ',masked_img.shape)

    # Save the masked image of each run
    print('Saving masked image of ',RUN)
    masked_img_fn = file_name_format + '_masked_fmri_img.nii.gz'
    nib.save(masked_img, f'{SUB_results_path}/{masked_img_fn}')

Now computing  sub-01_ses-01_task-01_run-1
Functional image shape:  (97, 115, 97, 600)
Brain mask image shape:  (97, 115, 97)
masked_image_data.shape:  (600, 268016)
masked_image.shape:  (97, 115, 97, 600)
Saving masked image of  run-1


### 1) Remove baseline volumes from the functional image and from the events df

In [13]:
runs = [1,2,3,4]

for run in runs:
    RUN = 'run-'+str(run)

    file_name_format = SUB + '_' + SES + '_' + TASK + '_'+ RUN 
    print('Now computing ',file_name_format)
    
    #Load of masked functional image (4D)
    masked_img_fn = os.path.join(results_path,SUB,file_name_format + '_masked_fmri_img.nii.gz')
    masked_img = nib.load(masked_img_fn)

    if labels_type == 'ptcps':    
      #Load of events dataframe with participant's labels
      events_df_fn=os.path.join(events_path, SUB+'_'+SES+'_'+TASK+'permusicparticipant_'+RUN+'_events.tsv')
      events_df=pd.read_csv(events_df_fn, sep='\t')

    elif labels_type == 'predef':
      #Load of events dataframe with predefined labels
      events_df_fn=os.path.join(events_path, SUB+'_'+SES+'_'+TASK+'permusic'+'_'+RUN+'_events.tsv')
      events_df=pd.read_csv(events_df_fn, sep='\t')
    
    # Load of movement confounds
   # movement_fp = os.path.join(data_path, 'func','rp_'+ file_name_format +'_bold.txt')
    #mov_param = np.loadtxt(movement_fp) 

    # Converter o ficheiro de texto para uma dataframe
    #mov_param_df = pd.DataFrame(mov_param, columns = ['tx','ty','tz','rx','ry','rz'])

    #Normalize image - z-score
    print('Normalising image...')
    #normalized_img = clean_img(masked_img, standardize='zscore', confounds=mov_param_df)
    normalized_img = clean_img(masked_img, standardize='zscore')


    #Remover volumes baseline da imagem funcional e da dataframe de eventos
    print('Removing baseline volumes...')
    fmri_img_nobaseline,events_nobaseline = remove_specificvols(normalized_img,events_df,'Baseline')
    print('Functional image without baseline volumes shape: ',fmri_img_nobaseline.shape)

    events_nobaseline = round(events_nobaseline)
    events_nobaseline = events_nobaseline.astype({'onset': int, 'duration': int})
    events_nobaseline['Run'] = RUN

    #Save image without baseline volumes of each run
    fmri_img_nobaseline_fn = file_name_format+'_nobaseline_fmri_VS.nii.gz'
    #nib.save(fmri_img_nobaseline, f'{SUB_results_path}/{fmri_img_nobaseline_fn}')

    #Save events dataframe without baseline volumes of each run
    events_nobaseline_fn = os.path.join(SUB_results_path,file_name_format+'_events_nobaseline_VS.csv')
    #events_nobaseline.to_csv(events_nobaseline_fn,sep=';')

    



Now computing  sub-01_ses-01_task-01_run-1
Normalising image...
Removing baseline volumes...
Functional image without baseline volumes shape:  (97, 115, 97, 384)


In [14]:
#Save image without baseline volumes of each run
fmri_img_nobaseline_fn = file_name_format+'_nobaseline_fmri_VS.nii.gz'
nib.save(fmri_img_nobaseline, f'{SUB_results_path}/{fmri_img_nobaseline_fn}')

#Save events dataframe without baseline volumes of each run
events_nobaseline_fn = os.path.join(SUB_results_path,file_name_format+'_events_nobaseline_VS.csv')
events_nobaseline.to_csv(events_nobaseline_fn,sep=';')

### 2) Fazer a média dos volumes noise e dos quadrantes a partir do 6º até ao ultimo

In [15]:
runs = [1,2,3,4]
for run in runs:
    RUN = 'run-'+str(run)

    file_name_format = SUB + '_' + SES + '_' + TASK + '_'+ RUN 
    print('Now computing ',file_name_format)
    
    #Load of func image without baseline volumes of each run
    fmri_img_nobaseline_fn =  os.path.join(SUB_results_path,file_name_format+'_nobaseline_fmri_VS.nii.gz')
    fmri_img_nobaseline = nib.load(fmri_img_nobaseline_fn)
    
    #Load of events dataframe without baseline volumes of each run
    events_nobaseline_fn = os.path.join(SUB_results_path,file_name_format+'_events_nobaseline_VS.csv')
    events_nobaseline = pd.read_csv(events_nobaseline_fn,sep=';',index_col=0)
    
    # Compute the mean volumes of each condition from the 6th volume to the last
    print('Computing the mean volumes of each condition...')
    fmri_img_meanvols = mean_xvolumes(events_nobaseline, fmri_img_nobaseline)

    #Guardar a imagem com as médias dos últimos volumes de cada trial type de cada run
    fmri_img_meanvols_fn = file_name_format+'_meanvols_fmri_VS.nii.gz'
    nib.save(fmri_img_meanvols, f'{SUB_results_path}/{fmri_img_meanvols_fn}')

Now computing  sub-01_ses-01_task-01_run-1
Computing the mean volumes of each condition...
Length idxs_tokeep_list:  32


### 3) Juntar todas as runs numa só imagem e dataframe

In [ ]:
#Inicialize the lists to join  all runs
imgs_runs_list = []
events_runs_list = []

for run in runs:
    RUN = 'run-'+str(run)

    file_name_format = SUB + '_' + SES + '_' + TASK + '_'+ RUN 
    print('Now computing ',file_name_format)

    #Load of events dataframe without baseline volumes of each run
    events_nobaseline_fn = os.path.join(SUB_results_path,file_name_format+'_events_nobaseline_VS.csv')
    events_nobaseline = pd.read_csv(events_nobaseline_fn,sep=';',index_col=0)

    #Load of func image with the mean volumes of each condition of each run
    fmri_img_meanvols_fn = os.path.join(SUB_results_path,file_name_format+'_meanvols_fmri_VS.nii.gz')
    fmri_img_meanvols = nib.load(fmri_img_meanvols_fn)

    #Append each run image and events dataframe to a list
    imgs_runs_list.append(fmri_img_meanvols)
    events_runs_list.append(events_nobaseline)


print('Concatenating the images of all runs...')
imgs_all_runs = concat_imgs(imgs_runs_list)
print('imgs_all_runs.shape: ',imgs_all_runs.shape)

print('Concatenating the events df of all runs...')
events_all_runs = pd.concat(events_runs_list,ignore_index=True)
print('events_all_runs.shape: ',events_all_runs.shape)

In [ ]:
## Save results ##
imgs_all_runs_fn = SUB + '_' + SES + '_' + TASK +'_allruns_VS.nii.gz'
nib.save(imgs_all_runs, f'{SUB_results_path}/{imgs_all_runs_fn}')

# Save dataframe with labels of all runs of each participant
events_all_runs_fn = os.path.join(SUB_results_path,SUB + '_' + SES + '_' + TASK + '_events_all_runs_VS.csv')
events_all_runs.to_csv(events_all_runs_fn,sep=';')
